In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import labrotation.file_handling as fh
import datadoc_util as ddutil
from labrotation import two_photon_session as tps
import h5py
from datetime import datetime as dt
from datetime import timedelta

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())


In [ ]:
SERVER_SYMBOL = env_dict["SERVER_SYMBOL"]  # symbol of server where data is stored
assert SERVER_SYMBOL is not None

In [ ]:
ddoc = ddutil.DataDocumentation(env_dict["DATA_DOCU_FOLDER"])
ddoc.loadDataDoc()

In [ ]:
matlab_2p_folder = env_dict["MATLAB_2P_FOLDER"]

In [ ]:
def get_datetime_for_fname():
    now = dt.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
events_list_fpath = os.path.join(env_dict["DATA_DOCU_FOLDER"], "events_list.xlsx")
assert os.path.exists(events_list_fpath)

In [ ]:
df_events = ddoc.getRecordingsWithExperimentType(["chr2_sd", "chr2_ctl", "chr2_szsd", "chr2_lfpsz_sd"])

In [ ]:
df_events = df_events[df_events["mouse_id"].isin(["T370", "T413"])]

In [ ]:
for uuid in df_events["uuid"].unique():
    recording_row = df_events[df_events["uuid"] == uuid].iloc[0]
    mouse_dir = recording_row.folder
    mouse_dir = SERVER_SYMBOL + mouse_dir[1:]
    if not os.path.exists(mouse_dir):
        print(uuid)
        print(f"\t{mouse_dir}")
        mouse_id = ddoc.getMouseIdForUuid(uuid)
        nik_name = ddoc.getNikonFileNameForUuid(uuid)
        print(f"\t{mouse_id} {nik_name}")
    else:
        has_lfp = True
        
        if type(recording_row.lfp) is float:
            has_lfp = False
        else:
            if not os.path.exists(os.path.join(mouse_dir,recording_row.lfp)):
                print(os.path.join(mouse_dir,recording_row.lfp))
            # assert )
            
        
        assert os.path.exists(os.path.join(mouse_dir, recording_row.nd2))
        
        assert os.path.exists(os.path.join(mouse_dir, recording_row.nikon_meta))
         
        assert os.path.exists(os.path.join(mouse_dir, recording_row.labview))
        assert os.path.exists(os.path.join(mouse_dir, recording_row.labview.replace(".txt", "time.txt")))
        
print("Consistency check done.")

In [ ]:
sessions_dict

In [ ]:
sessions_dict = dict()
has_lfp_dict = dict()

In [ ]:
for uuid in df_events["uuid"].unique():
    if uuid in sessions_dict.keys():
        print(f"{uuid} already opened. Skipping...")
        continue
    print(uuid)
    recording_row = df_events[df_events["uuid"] == uuid].iloc[0]
    session_folder = recording_row.folder
    session_folder = SERVER_SYMBOL + session_folder[1:]
    has_lfp = True
    if type(recording_row.lfp) is float:
        has_lfp = False
    else:
        if not os.path.exists(os.path.join(session_folder,recording_row.lfp)):
            print(os.path.join(session_folder,recording_row.lfp))
    files = ddoc.getSessionFilesForUuuid(uuid).iloc[0].to_dict()
    
    fname_nd2 = files["nd2"]
    fpath_nd2 = os.path.join(session_folder, fname_nd2)
    
    fname_nikmeta = files["nikon_meta"]
    fpath_nikmeta = os.path.join(session_folder, fname_nikmeta)
    
    fname_lv = files["labview"]
    fname_lvtime = os.path.splitext(files["labview"])[0] + "time.txt"
    fpath_lv = os.path.join(session_folder, fname_lv)
    fpath_lvtime = os.path.join(session_folder, fname_lvtime)
    
    fname_lfp = None
    fpath_lfp = None
    if has_lfp:
        fname_lfp = files["lfp"]
        fpath_lfp = os.path.join(session_folder, fname_lfp)
    
    session = tps.TwoPhotonSession.init_and_process(fpath_nd2, fpath_nikmeta, fpath_lv, fpath_lvtime, fpath_lfp, matlab_2p_folder)
    sessions_dict[uuid] = session
    has_lfp_dict[uuid] = session.has_lfp()

In [ ]:
has_lfp_dict

In [ ]:
def create_dt(t):
    t1 = t[1:]
    t0 = t[:-1]
    dt = np.zeros(len(t))
    dt[1:] = t1 - t0
    dt[0] = dt[1]  # assume same step size to avoid 0
    return dt
def create_totdist_abs(speed, dt):
    totdist_abs = np.zeros(len(speed))
    totdist_abs[0] = speed[0]*dt[0]
    for i in range(1, len(totdist_abs)):
        totdist_abs[i] = totdist_abs[i-1] + abs(speed[i]*dt[i])
    return totdist_abs

In [ ]:
traces_dict = dict()
for uuid in sessions_dict.keys():
    traces_dict[uuid] = dict()
    session = sessions_dict[uuid]
    if has_lfp_dict[uuid]:
        lfp_t, lfp_y = session.lfp_lfp()
        traces_dict[uuid]["lfp_t"] =  np.array(lfp_t)
        traces_dict[uuid]["lfp_y"] =  np.array(lfp_y)
        lfp_mov_t, lfp_mov_y = session.lfp_movement()
        traces_dict[uuid]["mov_t"] =  np.array(lfp_mov_t)
        traces_dict[uuid]["mov_y"] =  np.array(lfp_mov_y)
    else:
        traces_dict[uuid]["lfp_t"] =  None
        traces_dict[uuid]["lfp_y"] =  None
        traces_dict[uuid]["mov_t"] =  None
        traces_dict[uuid]["mov_y"] =  None
    lv_t_s = session.belt_scn_dict['tsscn']/1000.
    lv_dist = session.belt_scn_dict['distance']
    lv_speed = session.belt_scn_dict['speed']
    lv_running = session.belt_scn_dict['running']
    lv_totdist = session.belt_scn_dict['totdist']
    lv_rounds = session.belt_scn_dict['rounds']
    traces_dict[uuid]["lv_t_s"] = lv_t_s
    traces_dict[uuid]["lv_dist"] = lv_dist
    traces_dict[uuid]["lv_speed"] = lv_speed
    traces_dict[uuid]["lv_running"] = lv_running
    traces_dict[uuid]["lv_totdist"] = lv_totdist
    traces_dict[uuid]["lv_rounds"] = lv_rounds
    
    lv_dt = create_dt(lv_t_s)
    lv_totdist_abs = create_totdist_abs(lv_speed, lv_dt)
    traces_dict[uuid]["lv_dt"] = lv_dt
    traces_dict[uuid]["lv_totdist_abs"] = lv_totdist_abs
    
    mean_fluo = session.mean_fluo
    traces_dict[uuid]["mean_fluo"] = mean_fluo
    
    
    
        

In [ ]:
# TODO: create waterfall plots